In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
import pickle


In [2]:
# load the TCE list
tces = ascii.read("kplr_dr25_obs_tces.txt")
tces = tces.to_pandas()


In [3]:
# load the KOI list
dr25Koi = pd.read_csv("dr25_kois_archive.txt", dtype={"kepoi_name":str})

# add TCE_ID for comparison to the TCE list
TCE_ID = []
for i in range(len(dr25Koi)):
    # format the strings so the kepid part has 9 didgets with leading zeros 
    # and the planet number part has 2 didgets with leading zeros
    TCE_ID.append("%09d"%dr25Koi.kepid.loc[i] + "-%02d"%dr25Koi.koi_tce_plnt_num.loc[i])

dr25Koi["TCE_ID"] = TCE_ID


In [4]:
# look at TCEs that have score between 0.05 and 0.1
# highScoreTces = tces[(tces.Score > 0.05) & (tces.Score < 0.1)]
highScoreTces = tces[(tces.Score > 0.01) & (tces.Score < 0.05)]

In [5]:
highScoreTces

,TCE_ID,KIC,Disp,Score,NTL,SS,CO,EM,period,epoch,...,Rs,Ts,logg,a,Rp/Rs,a/Rs,impact,SNR_DV,Sp,Fit_Prov
89,001434395-01,1434395,FP,0.014,0,0,1,0,2.222722,131.900279,...,20.848,4996,2.354,0.00000,0.013365,6.64,0.0008,14.8,0.000,1
491,002308411-01,2308411,FP,0.014,0,0,1,0,0.983288,132.504131,...,1.198,6363,4.386,0.02098,0.008209,2.72,0.5765,10.9,4789.239,1
567,002438502-01,2438502,FP,0.046,0,1,0,0,8.360612,138.822956,...,2.445,5463,3.790,0.08900,0.150934,9.71,0.1914,501.6,602.311,1
654,002452185-04,2452185,FP,0.011,1,0,0,0,415.953886,421.126974,...,1.624,6704,4.126,1.18609,0.017439,294.45,0.7892,7.1,3.393,1
979,002716853-02,2716853,FP,0.040,1,0,1,0,68.831585,172.136601,...,0.860,5687,4.538,0.32104,0.023608,133.51,0.4256,8.1,6.725,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,011922290-01,11922290,FP,0.042,1,0,0,0,114.661574,237.231841,...,0.712,5412,4.562,0.40516,0.018141,167.79,0.8700,7.4,2.374,1
31554,011967004-01,11967004,FP,0.019,0,1,0,0,1.941641,133.107299,...,1.161,6525,4.381,0.03221,0.263831,6.58,0.7289,3480.7,2109.660,1
32039,012254688-01,12254688,FP,0.015,0,0,1,0,62.682423,182.230899,...,3.101,7058,3.648,0.35820,0.017342,283.54,0.3006,17.6,166.633,1
32075,012266450-01,12266450,FP,0.033,1,0,0,0,473.728882,306.311384,...,1.168,6875,4.376,1.25799,0.020796,737.23,0.5459,7.1,1.725,1


In [6]:
# are any KOIs?
highScoreTcesKois = dr25Koi[dr25Koi['TCE_ID'].isin(highScoreTces['TCE_ID'])]
highScoreTcesKois

,kepid,kepoi_name,kepler_name,ra,ra_err,ra_str,dec,dec_err,dec_str,koi_gmag,...,koi_fpflag_ec,koi_insol,koi_insol_err1,koi_insol_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_score,TCE_ID
284,11656840,K00774.01,NaN,288.98682,0.0,19h15m56.84s,49.733742,0.0,+49d44m01.5s,15.712,...,0,206.78,195.52,-61.76,7.48712,1.28308,-2.42034,LS+MCMC,0.014,011656840-01
440,2438502,K01003.01,NaN,290.32791,0.0,19h21m18.70s,37.726742,0.0,+37d43m36.3s,16.970,...,0,601.60,278.33,-356.79,0.24048,0.01736,-0.00068,LS+MCMC,0.046,002438502-01
588,2716853,K01022.02,NaN,292.26813,0.0,19h29m04.35s,37.955639,0.0,+37d57m20.3s,16.360,...,0,6.71,5.77,-1.84,8.38470,10.68587,-1.84046,LS+MCMC,0.040,002716853-02
698,2837111,K01110.02,NaN,287.44409,0.0,19h09m46.58s,38.088428,0.0,+38d05m18.3s,15.273,...,0,464.19,444.36,-141.70,0.65134,0.81468,-0.32978,LS+MCMC,0.047,002837111-02
766,5283542,K00827.01,NaN,294.22397,0.0,19h36m53.75s,40.417858,0.0,+40d25m04.3s,16.038,...,0,284.25,295.54,-90.93,0.14724,2.26962,-0.05173,LS+MCMC,0.037,005283542-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7713,1434395,K07623.01,NaN,291.85648,0.0,19h27m25.56s,37.065651,0.0,+37d03m56.3s,13.422,...,0,93172.12,12806.48,-67878.27,1.00737,0.29745,-0.56373,LS+MCMC,0.014,001434395-01
7825,2308411,K07631.01,NaN,292.63766,0.0,19h30m33.04s,37.641880,0.0,+37d38m30.8s,15.392,...,0,4774.53,3992.71,-1614.92,1.08608,0.20204,-1.05344,LS+MCMC,0.014,002308411-01
7860,7959310,K08150.01,NaN,291.54568,0.0,19h26m10.96s,43.789879,0.0,+43d47m23.6s,14.328,...,0,3.37,2.79,-1.95,0.37323,0.33689,-0.08968,LS+MCMC,0.022,007959310-01
7862,8028465,K08152.01,NaN,291.67062,0.0,19h26m40.95s,43.814461,0.0,+43d48m52.1s,15.863,...,0,0.22,0.06,-0.06,40.27950,4.22857,-37.45914,LS+MCMC,0.046,008028465-01


In [7]:
highScoreTcesKois.koi_pdisposition

284     FALSE POSITIVE
440     FALSE POSITIVE
588     FALSE POSITIVE
698     FALSE POSITIVE
766     FALSE POSITIVE
             ...      
7713    FALSE POSITIVE
7825    FALSE POSITIVE
7860    FALSE POSITIVE
7862    FALSE POSITIVE
7879    FALSE POSITIVE
Name: koi_pdisposition, Length: 86, dtype: object

In [8]:
highScoreTcesKois.koi_score

284     0.014
440     0.046
588     0.040
698     0.047
766     0.037
        ...  
7713    0.014
7825    0.014
7860    0.022
7862    0.046
7879    0.039
Name: koi_score, Length: 86, dtype: float64

In [9]:
# pull out the TCEs that are not KOIs
highScoreTcesNotKois = highScoreTces[~highScoreTces['TCE_ID'].isin(dr25Koi['TCE_ID'])]
highScoreTcesNotKois

,TCE_ID,KIC,Disp,Score,NTL,SS,CO,EM,period,epoch,...,Rs,Ts,logg,a,Rp/Rs,a/Rs,impact,SNR_DV,Sp,Fit_Prov
654,002452185-04,2452185,FP,0.011,1,0,0,0,415.953886,421.126974,...,1.624,6704,4.126,1.18609,0.017439,294.45,0.7892,7.1,3.393,1
1787,003330773-01,3330773,FP,0.012,1,0,0,0,0.990779,132.118014,...,0.740,4864,4.555,0.01741,0.005848,1.95,0.1997,10.3,905.851,1
1839,003340070-04,3340070,FP,0.036,0,0,1,0,403.699203,251.666599,...,0.867,5904,4.543,1.05378,0.030337,924.56,0.4154,7.6,0.737,1
2224,003526555-01,3526555,FP,0.036,1,0,0,0,456.072278,181.708454,...,0.709,5356,4.576,1.02522,0.027883,256.77,0.8869,7.1,0.353,1
2246,003532734-01,3532734,FP,0.011,1,0,0,0,49.672804,177.924453,...,5.544,4888,3.082,0.29263,0.015427,16.19,0.8676,8.6,183.575,1
3183,003940418-02,3940418,FP,0.021,1,0,0,0,397.771585,226.057985,...,0.807,5170,4.574,1.01870,0.024705,174.64,0.4214,8.7,0.402,1
3496,004060287-01,4060287,FP,0.036,1,0,0,0,270.624274,373.341968,...,0.839,5497,4.570,0.80622,0.033638,330.31,0.6114,9.0,0.886,1
4420,004476139-01,4476139,FP,0.015,1,0,0,0,174.344339,290.969339,...,0.836,6167,4.503,0.56987,0.020569,155.35,0.7829,7.4,2.789,1
4723,004640798-01,4640798,FP,0.019,1,0,0,0,409.737511,290.732716,...,0.889,5418,4.436,0.99676,0.019023,184.24,0.6995,7.1,0.614,1
5800,004988995-01,4988995,FP,0.011,1,0,0,0,256.805886,352.337052,...,1.327,5569,4.165,0.77449,0.014232,303.05,0.2942,7.1,2.530,1


In [10]:
list(highScoreTcesNotKois)

['TCE_ID',
 'KIC',
 'Disp',
 'Score',
 'NTL',
 'SS',
 'CO',
 'EM',
 'period',
 'epoch',
 'Expected_MES',
 'MES',
 'NTran',
 'depth',
 'duration',
 'Rp',
 'Rs',
 'Ts',
 'logg',
 'a',
 'Rp/Rs',
 'a/Rs',
 'impact',
 'SNR_DV',
 'Sp',
 'Fit_Prov']

In [11]:
highScoreTcesNotKois.to_csv("midScoreTcesNotKois.csv", 
              columns=["TCE_ID", "Score", "period", "Rp", "SNR_DV"],
             index=False)


In [14]:
print((highScoreTcesNotKois['period']).to_string(index=False))

 415.953886
   0.990779
 403.699203
 456.072278
  49.672804
 397.771585
 270.624274
 174.344339
 409.737511
 256.805886
   5.239758
 246.583120
  72.079069
 231.901920
 137.308666
 362.924753
 283.604542
 350.849388
 367.404274
 280.203243
 186.504031
 216.193703
 171.432610
 313.299571
 226.637011
 433.030127
 382.321313
 281.838618
 251.643226
 323.686409
  86.408921
 351.650750
 376.767778
 507.753753
 258.722122
  99.257541
 401.331951
 205.110578
 277.077402
  42.988070
 284.445032
 151.690028
 237.238351
 334.038428
   1.348916
 195.941873
 424.793826
 471.620861
 524.753574
 103.302853
 118.726252
 467.000095
 366.853683
  38.926088
 127.765806
 285.782952
 473.728882
 340.331273
